In [1]:
from RAG_correctors import repair_hard_wrongs
import pandas as pd
import numpy as np
import importlib

In [ ]:
df = pd.read_csv("RAG_final_thesis_results.csv")

In [3]:
print(df.shape)
print(df.columns)
print(df.dtypes)

# Check for required columns
required_cols = ['llm_verdict', 'rag_start_lat', 'rag_start_lon', 'rag_end_lat', 'rag_end_lon', 'transportmiddel']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"Missing required columns: {missing_cols}")
else:
    print("All required columns present.")

(234531, 39)
Index(['SessionId', 'homeText_raw', 'workText_raw', 'startDayText_raw',
       'startstedadrsogeord', 'startTripText_raw', 'turid', 'tiladrsogeord',
       'tiladrtext_raw', 'transportmiddel', 'stagelength_raw',
       'stagedurationmin_raw', 'home_lat', 'home_lon', 'work_lat', 'work_lon',
       'start_lat', 'start_lon', 'til_lat', 'til_lon', 'calc_dist_geo',
       'dist_start_home', 'dist_end_home', 'flag_coords_bad', 'flag_speed_bad',
       'flag_zero_dist', 'validation_status', 'prompt_text',
       'rag_start_address', 'rag_start_lat', 'rag_start_lon', 'rag_start_conf',
       'rag_end_address', 'rag_end_lat', 'rag_end_lon', 'rag_end_conf',
       'rag_prompt_text', 'llm_full_response', 'llm_verdict'],
      dtype='object')
SessionId                 int64
homeText_raw             object
workText_raw             object
startDayText_raw         object
startstedadrsogeord      object
startTripText_raw        object
turid                     int64
tiladrsogeord         

In [4]:
# Debug: Test OSRM on a sample invalid row
from RAG_correctors import get_osrm_route_bridge

# Find an invalid row
invalid_rows = df[df['llm_verdict'] == 'INVALID']
if not invalid_rows.empty:
    sample_row = invalid_rows.iloc[0]
    print("Sample row values:")
    print(f"Start: {sample_row['rag_start_lat']}, {sample_row['rag_start_lon']}")
    print(f"End: {sample_row['rag_end_lat']}, {sample_row['rag_end_lon']}")
    print(f"Transport: {sample_row['transportmiddel']} (type: {type(sample_row['transportmiddel'])})")
    print(f"NaN in coords: {pd.isna(sample_row[['rag_start_lat', 'rag_start_lon', 'rag_end_lat', 'rag_end_lon']]).any()}")
    print(f"Transport NaN: {pd.isna(sample_row['transportmiddel'])}")
    
    # Test OSRM call
    result = get_osrm_route_bridge(
        sample_row['rag_start_lat'], 
        sample_row['rag_start_lon'], 
        sample_row['rag_end_lat'], 
        sample_row['rag_end_lon'], 
        sample_row['transportmiddel']
    )
    print(f"OSRM result: {result}")
else:
    print("No invalid rows found.")

Sample row values:
Start: 54.66261819, 11.35739183
End: 55.8001054, 10.55570583
Transport: 11.0 (type: <class 'numpy.float64'>)
NaN in coords: False
Transport NaN: False
OSRM result: (211.6, np.float64(230.0))


In [5]:
corrected_df = repair_hard_wrongs(df)

print("Original invalid count:", (df['llm_verdict'] == 'INVALID').sum())
print("Corrected count:", (corrected_df['llm_verdict'] == 'corrected').sum())

--- Starting Correction Pipeline on 8067 rows ---


Correcting rows:   3%|▎         | 204/8067 [00:36<11:41, 11.21it/s] 

Processed 200 rows, fixed 196 so far.


Correcting rows:   5%|▍         | 398/8067 [00:56<16:23,  7.80it/s]

Processed 400 rows, fixed 396 so far.


...

Processed 7800 rows, fixed 7784 so far.


Correcting rows:  99%|█████████▉| 8006/8067 [04:27<00:02, 29.12it/s]

Processed 8000 rows, fixed 7984 so far.


Correcting rows: 100%|██████████| 8067/8067 [04:28<00:00, 30.00it/s]


--- Finished. Corrected 8051 rows. OSRM attempted: 8067 ---
Original invalid count: 8067
Corrected count: 8051


In [6]:
corrected_df.columns

Index(['SessionId', 'homeText_raw', 'workText_raw', 'startDayText_raw',
       'startstedadrsogeord', 'startTripText_raw', 'turid', 'tiladrsogeord',
       'tiladrtext_raw', 'transportmiddel', 'stagelength_raw',
       'stagedurationmin_raw', 'home_lat', 'home_lon', 'work_lat', 'work_lon',
       'start_lat', 'start_lon', 'til_lat', 'til_lon', 'calc_dist_geo',
       'dist_start_home', 'dist_end_home', 'flag_coords_bad', 'flag_speed_bad',
       'flag_zero_dist', 'validation_status', 'prompt_text',
       'rag_start_address', 'rag_start_lat', 'rag_start_lon', 'rag_start_conf',
       'rag_end_address', 'rag_end_lat', 'rag_end_lon', 'rag_end_conf',
       'rag_prompt_text', 'llm_full_response', 'llm_verdict',
       'corrected_dist_km', 'corrected_time_min', 'note_correction'],
      dtype='object')

In [7]:
df['corrected_dist_km'] = np.nan
df['corrected_time_min'] = np.nan
df['note_correction'] = np.nan

df.update(corrected_df)

C:\Users\nicoc\AppData\Local\Temp\ipykernel_17672\4145188863.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.update(corrected_df)


In [ ]:
df.to_csv('RAG_corrected_df.csv', index=False)
print("Corrected data saved to 'RAG_corrected_df.csv'")

Corrected data saved to 'RAG_corrected_df_v1.csv'


In [9]:
df.columns

Index(['SessionId', 'homeText_raw', 'workText_raw', 'startDayText_raw',
       'startstedadrsogeord', 'startTripText_raw', 'turid', 'tiladrsogeord',
       'tiladrtext_raw', 'transportmiddel', 'stagelength_raw',
       'stagedurationmin_raw', 'home_lat', 'home_lon', 'work_lat', 'work_lon',
       'start_lat', 'start_lon', 'til_lat', 'til_lon', 'calc_dist_geo',
       'dist_start_home', 'dist_end_home', 'flag_coords_bad', 'flag_speed_bad',
       'flag_zero_dist', 'validation_status', 'prompt_text',
       'rag_start_address', 'rag_start_lat', 'rag_start_lon', 'rag_start_conf',
       'rag_end_address', 'rag_end_lat', 'rag_end_lon', 'rag_end_conf',
       'rag_prompt_text', 'llm_full_response', 'llm_verdict',
       'corrected_dist_km', 'corrected_time_min', 'note_correction'],
      dtype='object')